# ASTR3110 Tutorial 6: Clustering with Gaussian Mixture Models

Tutorial 7 of the *'Data Science Techniques in Astrophysics'* course at Macquarie University.

## Learning outcomes from this tutorial

 * Understand the basic Gaussian Mixture modelling technique.
 * Use the Scikit Learn clustering algorithm to find clusters in 2D data.
 * Test the behaviour GMM clustering with different assumptions.

## Setup

This week, we won't need to access any data on disk, so simply start a new *Python 3* notebook on Google Colab.The tutorial content is based on a section of the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas. Additional content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).

## Introduction to GMMs

In this lectorial we will take a look at Gaussian mixture models (GMMs), which can be viewed as an extension of the ideas behind *k*-means, but can also be a powerful tool for estimation beyond simple clustering.

As we saw in Week 6, given simple, well-separated data, *k*-means finds suitable clustering results.
One way to think about the k-means model is that it places a circle (or, in higher dimensions, a hyper-sphere) at the center of each cluster, with a radius defined by the most distant point in the cluster. This radius acts as a hard cutoff for cluster assignment within the training set: any point outside this circle is not considered a member of the cluster.

In [1]:
# Import necessary modules and set plots to appear inline.


In [2]:
# Generate some data


In [3]:
# Plot the data with K Means Labels


One way to think about the k-means model is that it places a circle (or, in higher dimensions, a hyper-sphere) at the center of each cluster, with a radius defined by the most distant point in the cluster. This radius acts as a hard cutoff for cluster assignment within the training set: any point outside this circle is not considered a member of the cluster. We can visualize this cluster model with the following function:

In [6]:
from scipy.spatial.distance import cdist

def plot_kmeans(myKM, data, n_clusters=4, rseed=0, ax=None):
    clusters = myKM.fit_predict(data)

    # Plot the input data
    ax = ax or plt.gca()#gca= get current axis (in case we are overplotting onto an exiting axis)
    ax.axis('equal')
    ax.scatter(data[:, 0], data[:, 1], c=clusters, s=40, cmap='jet', zorder=2)

    # Plot the representation of the KMeans model
    centers = myKM.cluster_centers_
    #Use cdist to determine the maximum radial extent of the data points allocated to each cluster.
    #cdist takes the x,y data points and measures distance from center
    radii = [cdist(data[clusters == i], [center]).max() 
             for i, center in enumerate(centers)]
    #recall zip takes two vectors and pairs them by row number as [(centers[0],radii[0]), (centers[1],radii[1])...]
    for c, r in zip(centers, radii):
        #add_patch lets you add different shapes onto an image. Can add circles, ellipses, triangles etc.
        #fc=face colour, ex=edge colour, zorder tells order to plot. 
        ax.add_patch(plt.Circle(c, r, fc='#CCCCCC', ec='k', lw=3, alpha=0.5, zorder=1)) 

An important observation for *k*-means is that these cluster models *must be circular*: *k*-means has no built-in way of accounting for oblong or elliptical clusters.
So, for example, if we take the same data and transform it, the cluster assignments end up becoming muddled:

In [7]:
# Stretch the Y-axis

## Generalizing Expectation–Maximisation: Gaussian Mixture Models

A Gaussian mixture model (GMM) attempts to find a mixture of multi-dimensional Gaussian probability distributions that best model any input dataset.
In the simplest case, GMMs can be used for finding clusters in the same manner as *k*-means:

But because GMM contains a probabilistic model under the hood, it is also possible to find probabilistic cluster assignments—in Scikit-Learn this is done using the ``predict_proba`` method.
This returns a matrix of size ``[n_samples, n_clusters]`` which measures the probability that any point belongs to the given cluster:

Under the hood, a Gaussian mixture model is very similar to *k*-means: it uses an expectation–maximization approach which qualitatively does the following:

1. Choose starting guesses for the location and shape of each Gaussian component (AKA cluster)

2. Repeat until converged:

   1. *E-step*: for each point, find weights encoding the probability of membership in each cluster
   2. *M-step*: for each cluster, update its location, normalization, and shape based on *all* data points, making use of the weights

The result of this is that each cluster is associated not with a hard-edged sphere, but with a smooth Gaussian model.
Just as in the *k*-means expectation–maximization approach, this algorithm can sometimes miss the globally optimal solution, and thus in practice multiple random initializations are used.

Let's create a function that will help us visualize the locations and shapes of the GMM clusters by drawing ellipses based on the GMM output:

In [9]:
#this is similar to the above function that adds a circle to a plot, but is now
#generalised to an ellipse, which can have a set ellipticity and position angle
#that comes from the covariance.
from matplotlib.patches import Ellipse

def draw_ellipse(position, covariance, covariance_type, ax=None, **kwargs):
    """Draw an ellipse with a given position and covariance"""
    
    
    ax = ax or plt.gca()#gca= get current axis (in case we are overplotting onto an exiting axis)
    
    # Convert covariance to principal axes
    if covariance_type == 'full':#shape == (2, 2):
        #the below three lines of code convert the covariance matrix to a position angle, 
        #and two lengths for the minor and major axis of the ellipse
        U, s, Vt = np.linalg.svd(covariance)
        angle = np.degrees(np.arctan2(U[1, 0], U[0, 0]))
        width, height = 2 * np.sqrt(s)
    elif covariance_type == 'diag':
        angle = 0
        width, height = 2 * np.sqrt(covariance)
    elif covariance_type == 'spherical':
        angle = 0
        width = 2 * np.sqrt(covariance)
        height = 2 * np.sqrt(covariance)
    # Draw ellipses at one-sigma, two-sigma, three-sigma and four-sigma.
    for nsig in range(1, 4):
        ax.add_patch(Ellipse(position, nsig * width, nsig * height,
                             angle, **kwargs))

In [10]:
#function that will take in gmm and data and make a scatterplot of data with ellipses overlaid
def plot_gmm(gmm, X, label=True, ax=None):
    ax = ax or plt.gca()
    labels = gmm.fit(X).predict(X)
    if label:
        ax.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis', zorder=2)
    else:
        ax.scatter(X[:, 0], X[:, 1], s=40, zorder=2)
    ax.axis('equal')
    
    w_factor = 0.2 / gmm.weights_.max()
    for pos, covar, w in zip(gmm.means_, gmm.covariances_, gmm.weights_):
        print(pos.shape, covar.shape)
        draw_ellipse(pos, covar, gmm.covariance_type, alpha=w * w_factor, ec='k')

Similarly, we can use the GMM approach to fit our stretched dataset; allowing for a full covariance the model will fit even very oblong, stretched-out clusters:

### Choosing the covariance type

If you look at the details for Gaussian Mixture, you will see that the ``covariance_type`` option can be set to different types.
This hyperparameter controls the degrees of freedom in the shape of each cluster; it is essential to set this carefully for any given problem.
Setting the ``covariance_type="diag"`` means that the size of the cluster along each dimension can be set independently, with the resulting ellipse constrained to align with the axes.
A slightly simpler and faster model is ``covariance_type="spherical"``, which constrains the shape of the cluster such that all dimensions are equal. The resulting clustering will have similar characteristics to that of *k*-means, though it is not entirely equivalent.
A more complicated and computationally expensive model (especially as the number of dimensions grows) is to use ``covariance_type="full"``, which allows each cluster to be modeled as an ellipse with arbitrary orientation.

We can see how this works using our two example datasets:

## GMM as *Density Estimation*

Though GMM is often categorized as a clustering algorithm, fundamentally it is an algorithm for *density estimation*.
That is to say, the result of a GMM fit to some data is technically not a clustering model, but a generative probabilistic model describing the distribution of the data.

As an example, consider some data generated from Scikit-Learn's ``make_moons`` function, which generates crescent-moon shaped distribtutions:

In [ ]:
#try 2 components

In [ ]:
#try 16 components

Here the mixture of 16 Gaussians serves not to find separated clusters of data, but rather to model the overall *distribution* of the input data.
This is a generative model of the distribution, meaning that the GMM gives us the recipe to generate new random data distributed similarly to our input.
For example, here are 400 new points drawn from this 16-component GMM fit to our original data:

### How many components?

In the Week 3 lectorial, we met the problem of trying to ascertain the optimum number of polynomial parameters that are required to fit the data. Optimal here meant that we want the minimum polynomial order that describes the data without over-fitting (i.e., fitting fluctuations caused by uncertainties in the data). 

We have a similar problem in attempting to understand how many clusters/components are required to oprtimally fit the data. Again, we can use the the [Bayesian information criterion (BIC)](https://en.wikipedia.org/wiki/Bayesian_information_criterion), which includes the likelihood measured for the fit along with a term that penalises an increase in the number of components to avoid over-fitting. Scikit-Learn's ``GMM`` estimator actually includes built-in methods that compute BIC (and another information criteria called the Akike Information Criteria), and so it is very easy to operate on this approach.

Let's look at the BIC as a function as the number of GMM components for our moon dataset:

In [25]:
myGMM?